# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-b39a9416e4-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Tennis explorative search") 

Consider the following exploratory information need:

> investigate tennis and find the main BGPs related to this sport. Compare players per countries, awards of the players and tournament.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q1426`| Roger Federer       | node |
| `wd:Q11459`| Serena Williams       | node |





Also consider

```
wd:Q1426 ?p ?obj .
```

is the BGP to retrieve all **properties of Roger Federer**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for tennis player

2. Identify the BGP for tennis tournament

3. Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

4. Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

5. Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners (hint: men and women do not play together, there are two distinct tournament ) (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

6. Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

7. Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

8. Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

## Task 1
Identify the BGP for tennis player

In [2]:
# write your queries
# getting object/data properties of node 'Roger Federer'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q1426 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1006'), ('label', 'Nationale Thesaurus voor Auteurs ID')]
[('property', 'http://www.wikidata.org/prop/direct/P103'), ('label', 'native language')]
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('label', 'occupation')]
[('property', 'http://www.wikidata.org/prop/direct/P108'), ('label', 'employer')]
[('property', 'http://www.wikidata.org/prop/direct/P1263'), ('label', 'NNDB people ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1280'), ('label', 'CONOR.SI ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1284'), ('label', 'Munzinger person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1285'), ('label', 'Munzinger Sport number')]
[('property', 'http://www.wikidata.org/prop/direct/P1296'), ('label', 'Gran Enciclopèdia Catalana ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P1344'), ('l

In [3]:
# checking 'occupation' property of node 'Roger Federer'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q1426 wdt:P106 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q10833314'), ('label', 'tennis player')]
1


Final query for this task

In [4]:
# write your final query
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q1426 wdt:P106 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q10833314'), ('label', 'tennis player')]
1


## Task 2
Identify the BGP for tennis tournament

In [5]:
# write your queries
# checking 'participant in' property of node 'Roger Federer'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q1426 wdt:P1344 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q589310'), ('label', "tennis at the 2000 Summer Olympics – men's singles")]
[('object', 'http://www.wikidata.org/entity/Q8544'), ('label', '2000 Summer Olympics')]
[('object', 'http://www.wikidata.org/entity/Q8558'), ('label', '2004 Summer Olympics')]
[('object', 'http://www.wikidata.org/entity/Q581788'), ('label', "tennis at the 2012 Summer Olympics – men's singles")]
[('object', 'http://www.wikidata.org/entity/Q757054'), ('label', "tennis at the 2008 Summer Olympics – men's doubles")]
5


In [6]:
# getting object/data properties of node 'Q589310' retrieved from last query output
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q589310 wdt:P31 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q18536594'), ('label', 'Olympic sporting event')]
[('object', 'http://www.wikidata.org/entity/Q46190676'), ('label', 'tennis event')]
2


In [7]:
# getting object/data properties of node 'tennis event'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q46190676 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('label', "topic's main template")]
[('property', 'http://www.wikidata.org/prop/direct/P1552'), ('label', 'has quality')]
[('property', 'http://www.wikidata.org/prop/direct/P1963'), ('label', 'properties for this type')]
[('property', 'http://www.wikidata.org/prop/direct/P2670'), ('label', 'has parts of the class')]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('property', 'http://www.wikidata.org/prop/direct/P361'), ('label', 'part of')]
[('property', 'http://www.wikidata.org/prop/direct/P641'), ('label', 'sport')]
7


In [8]:
# s:'tennis event' p:'part of' o:what node?
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q46190676 wdt:P361 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q47345468'), ('label', 'tennis tournament edition')]
1


In [9]:
# getting object/data properties of node 'tennis tournament edition' 
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q47345468 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('label', "topic's main template")]
[('property', 'http://www.wikidata.org/prop/direct/P1963'), ('label', 'properties for this type')]
[('property', 'http://www.wikidata.org/prop/direct/P2670'), ('label', 'has parts of the class')]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('property', 'http://www.wikidata.org/prop/direct/P361'), ('label', 'part of')]
[('property', 'http://www.wikidata.org/prop/direct/P641'), ('label', 'sport')]
[('property', 'http://www.wikidata.org/prop/direct/P8408'), ('label', 'KBpedia ID')]
7


In [10]:
# s:'tennis tournament edition' p:'part of' o:what node?
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q47345468 wdt:P361 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q13219666'), ('label', 'tennis tournament')]
[('object', 'http://www.wikidata.org/entity/Q47358534'), ('label', 'tennis tour edition')]
2


Final query for this task

In [11]:
# write your final query
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q1426 wdt:P1344 wd:Q589310.
   wd:Q589310 wdt:P31 wd:Q46190676.
   wd:Q46190676 wdt:P361 wd:Q47345468.
   wd:Q47345468 wdt:P361 ?object.
   
   ?object sc:name ?label.
}
LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q13219666'), ('label', 'tennis tournament')]
1


## Task 3
Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

In [12]:
# write your queries
# checking the 'victory' property of node 'Roger Federer'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q1426 wdt:P2522 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q22956224'), ('label', "2017 Australian Open – Men's Singles")]
[('object', 'http://www.wikidata.org/entity/Q612597'), ('label', '2007 Tennis Masters Cup')]
[('object', 'http://www.wikidata.org/entity/Q782589'), ('label', "2010 Australian Open – Men's Singles")]
[('object', 'http://www.wikidata.org/entity/Q782420'), ('label', "2006 Australian Open – Men's Singles")]
[('object', 'http://www.wikidata.org/entity/Q300024'), ('label', '2011 ATP World Tour Finals')]
[('object', 'http://www.wikidata.org/entity/Q753300'), ('label', "2012 Wimbledon Championships – Gentlemen's Singles")]
[('object', 'http://www.wikidata.org/entity/Q757054'), ('label', "tennis at the 2008 Summer Olympics – men's doubles")]
[('object', 'http://www.wikidata.org/entity/Q1074654'), ('label', "2006 Wimbledon Championships – Men's Singles")]
[('object', 'http://www.wikidata.org/entity/Q384929'), ('label', "2005 Wimbledon Championships – Men's Singles")]
[('object', 'h

In [13]:
# getting object/data properties of node '2012 wimbledon championships – gentlemen's singles' 
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q753300 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1346'), ('label', 'winner')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('label', 'country')]
[('property', 'http://www.wikidata.org/prop/direct/P2094'), ('label', 'competition class')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P361'), ('label', 'part of')]
[('property', 'http://www.wikidata.org/prop/direct/P585'), ('label', 'point in time')]
[('property', 'http://www.wikidata.org/prop/direct/P641'), ('label', 'sport')]
[('property', 'http://www.wikidata.org/prop/direct/P646'), ('label', 'Freebase ID')]
8


In [14]:
# checking the 'part of' property of node '2012 wimbledon championships – gentlemen's singles'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q753300 wdt:P361 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q289587'), ('label', '2012 Wimbledon Championships')]
1


In [15]:
# getting object/data properties of node '2012 wimbledon championships'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q289587 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P6880'), ('label', 'ATP tennis tournament edition ID')]
[('property', 'http://www.wikidata.org/prop/direct/P155'), ('label', 'follows')]
[('property', 'http://www.wikidata.org/prop/direct/P156'), ('label', 'followed by')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('label', 'country')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P2581'), ('label', 'BabelNet ID')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P393'), ('label', 'edition number')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('label', 'has part')]
[('property', 'http://www.wikidata.org/prop/direct/P580'), ('label', 'start time')]
[('property', 'http://www.wikidata.org/prop/dir

In [16]:
# checking the 'instance of' property of node '2012 wimbledon championships'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q289587 wdt:P31 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q47345468'), ('label', 'tennis tournament edition')]
[('object', 'http://www.wikidata.org/entity/Q41520'), ('label', 'Wimbledon Championships')]
2


In [17]:
# getting object/data properties of node 'wimbledon championships'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q41520 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P131'), ('label', 'located in the administrative territorial entity')]
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('label', "topic's main template")]
[('property', 'http://www.wikidata.org/prop/direct/P1687'), ('label', 'Wikidata property')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('label', 'country')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('label', 'different from')]
[('property', 'http://www.wikidata.org/prop/direct/P2002'), ('label', 'Twitter username')]
[('property', 'http://www.wikidata.org/prop/direct/P2003'), ('label', 'Instagram username')]
[('property', 'http://www.wikidata.org/prop/direct/P2013'), ('label', 'Facebook ID')]
[('property', 'http://www.wikidata.org/prop/direct/P214')

In [18]:
# checking the 'part of' property of node 'wimbledon championships'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q41520 wdt:P361 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q102113'), ('label', 'Grand Slam of tennis')]
1


In [19]:
# getting object/data properties of node 'grand slam of tennis'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q102113 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('label', "topic's main template")]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P2354'), ('label', 'has list')]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('property', 'http://www.wikidata.org/prop/direct/P2924'), ('label', 'Great Russian Encyclopedia Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P3222'), ('label', 'NE.se ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3417'), ('label', 'Quora topic ID')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('label', 'has part')]
[('

In [20]:
# checking the 'has list' property of node 'grand slam of tennis'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q102113 wdt:P2354 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q1426439'), ('label', "list of Grand Slam men's singles champions")]
[('object', 'http://www.wikidata.org/entity/Q645455'), ('label', "list of Grand Slam women's singles champions")]
2


In [21]:
# checking the 'has part' property of node 'grand slam of tennis'

# one of the outputs is 'french cheese' instead of being 'french open' which is a weird problem! 
# so I checked wikidata online and the instance is correctly written as 'french open' on there
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q102113 wdt:P527 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q2223649'), ('label', 'French cheese')]
[('object', 'http://www.wikidata.org/entity/Q123577'), ('label', 'US Open')]
[('object', 'http://www.wikidata.org/entity/Q41520'), ('label', 'Wimbledon Championships')]
[('object', 'http://www.wikidata.org/entity/Q60874'), ('label', 'Australian Open')]
4


Final query for this task

In [ ]:
# write your final query
# takes around 4/5 minutes to output the result
 
# the output being 'french cheese' will probably affect the results of the other tasks!
queryString = """
SELECT ?grand_slam_iri ?label
WHERE
{
   wd:Q1426 wdt:P2522 wd:Q753300.
   wd:Q753300 wdt:P361 wd:Q289587.
   wd:Q289587 wdt:P31 wd:Q41520.
   wd:Q41520 wdt:P361 wd:Q102113.
   wd:Q102113 wdt:P527 ?grand_slam_iri.
   
   ?grand_slam_iri sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

## Task 4
Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

In [24]:
# write your queries
# checking 'occupation' property of node 'Roger Federer'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q1426 wdt:P106 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q10833314'), ('label', 'tennis player')]
1


In [26]:
# getting object/data properties of node 'tennis player'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q10833314 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1014'), ('label', 'Art & Architecture Thesaurus ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1296'), ('label', 'Gran Enciclopèdia Catalana ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('label', "topic's main template")]
[('property', 'http://www.wikidata.org/prop/direct/P1552'), ('label', 'has quality')]
[('property', 'http://www.wikidata.org/prop/direct/P1709'), ('label', 'equivalent class')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P1963'), ('label', 'properties for this type')]
[('property', 'http://www.wikidata.org/prop/direct/P227'), ('label', 'GND ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2354'), ('label', 'has list')]
[('property', 'http://www.wikidata.org/prop/direct/P244'), ('label', 'Library of Congress authority ID')]
[('property', 'http://www.wikidata.org/prop/direct/P268'), ('la

In [27]:
# checking 'sport' property of node 'tennis player'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q10833314 wdt:P641 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q847'), ('label', 'tennis')]
1


In [28]:
# getting object/data properties of node 'tennis'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q847 ?property ?object.
   
   ?property sc:name ?label.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1014'), ('label', 'Art & Architecture Thesaurus ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1051'), ('label', 'PSH ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1151'), ('label', "topic's main Wikimedia portal")]
[('property', 'http://www.wikidata.org/prop/direct/P1225'), ('label', 'U.S. National Archives Identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P1245'), ('label', 'OmegaWiki Defined Meaning')]
[('property', 'http://www.wikidata.org/prop/direct/P1256'), ('label', 'Iconclass notation')]
[('property', 'http://www.wikidata.org/prop/direct/P1282'), ('label', 'OpenStreetMap tag or key')]
[('property', 'http://www.wikidata.org/prop/direct/P1296'), ('label', 'Gran Enciclopèdia Catalana ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online I

In [29]:
# checking 'uses' property of node 'tennis'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q847 wdt:P2283 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q153362'), ('label', 'tennis racquet')]
[('object', 'http://www.wikidata.org/entity/Q741118'), ('label', 'tennis court')]
[('object', 'http://www.wikidata.org/entity/Q246514'), ('label', 'tennis ball')]
3


In [30]:
# getting object/data properties of node 'tennis court'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q741118 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1225'), ('label', 'U.S. National Archives Identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P1282'), ('label', 'OpenStreetMap tag or key')]
[('property', 'http://www.wikidata.org/prop/direct/P1535'), ('label', 'used by')]
[('property', 'http://www.wikidata.org/prop/direct/P1552'), ('label', 'has quality')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P2581'), ('label', 'BabelNet ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2670'), ('label', 'has parts of the class')]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('property', 'http://www.wikidata.org/prop/direct/P3417'), ('label', 'Quora topic ID')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P3827'), ('label', 'JSTOR topic ID')]
[('

In [31]:
# checking 'has parts of the class' property of node 'tennis court'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q741118 wdt:P2670 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q47677460'), ('label', 'tennis playing surface')]
[('object', 'http://www.wikidata.org/entity/Q48291351'), ('label', 'facet of tennis playing area')]
[('object', 'http://www.wikidata.org/entity/Q4839569'), ('label', 'backboard')]
3


In [32]:
# getting object/data properties of node 'tennis playing surface'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q47677460 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1687'), ('label', 'Wikidata property')]
[('property', 'http://www.wikidata.org/prop/direct/P2671'), ('label', 'Google Knowledge Graph ID')]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('property', 'http://www.wikidata.org/prop/direct/P361'), ('label', 'part of')]
[('property', 'http://www.wikidata.org/prop/direct/P641'), ('label', 'sport')]
[('property', 'http://www.wikidata.org/prop/direct/P8408'), ('label', 'KBpedia ID')]
[('property', 'http://www.wikidata.org/prop/direct/P910'), ('label', "topic's main category")]
7


In [33]:
# checking 'part of' property of node 'tennis playing surface'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q47677460 wdt:P279 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q28840755'), ('label', 'surface')]
1


In [34]:
# checking 'subclass of' property of node 'tennis playing surface'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q47677460 wdt:P279 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q28840755'), ('label', 'surface')]
1


In [35]:
# getting object/data properties of node 'US open' as found from last task's final query
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q123577 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P154'), ('label', 'logo image')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('label', 'country')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P2002'), ('label', 'Twitter username')]
[('property', 'http://www.wikidata.org/prop/direct/P2003'), ('label', 'Instagram username')]
[('property', 'http://www.wikidata.org/prop/direct/P2013'), ('label', 'Facebook ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2121'), ('label', 'prize money')]
[('property', 'http://www.wikidata.org/prop/direct/P214'), ('label', 'VIAF ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2257'), ('label', 'event interval')]
[('property', 'http://www.wikidata.org/prop/direct/P2354'), ('label', 'has list')]
[('property', 'http://www.wikidat

In [36]:
# checking 'surface played on' property of node 'US open'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q123577 wdt:P765 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q1330871'), ('label', 'hardcourt')]
1


In [37]:
# checking 'surface played on' property of node 'wimbledon championships'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q41520 wdt:P765 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q1151291'), ('label', 'grass court')]
1


Final query for this task

In [38]:
# write your final query

# the 'french open' grand slam has not been considered in query 
# because of the problem with task 3, that 'french cheese' does not have the property 'surface played on'
queryString = """
SELECT DISTINCT ?surface_iri ?label
WHERE
{
   wd:Q102113 wdt:P527 ?grand_slam.
   ?grand_slam wdt:P765 ?surface_iri.
   
   ?surface_iri sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('surface_iri', 'http://www.wikidata.org/entity/Q1330871'), ('label', 'hardcourt')]
[('surface_iri', 'http://www.wikidata.org/entity/Q1151291'), ('label', 'grass court')]
2


## Task 5
Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners (hint: men and women do not play together, there are two distinct tournament ) (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

In [39]:
# checking 'victory' property of node 'Roger Federer'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q1426 wdt:P2522 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q22956224'), ('label', "2017 Australian Open – Men's Singles")]
[('object', 'http://www.wikidata.org/entity/Q612597'), ('label', '2007 Tennis Masters Cup')]
[('object', 'http://www.wikidata.org/entity/Q782589'), ('label', "2010 Australian Open – Men's Singles")]
[('object', 'http://www.wikidata.org/entity/Q782420'), ('label', "2006 Australian Open – Men's Singles")]
[('object', 'http://www.wikidata.org/entity/Q300024'), ('label', '2011 ATP World Tour Finals')]
[('object', 'http://www.wikidata.org/entity/Q753300'), ('label', "2012 Wimbledon Championships – Gentlemen's Singles")]
[('object', 'http://www.wikidata.org/entity/Q757054'), ('label', "tennis at the 2008 Summer Olympics – men's doubles")]
[('object', 'http://www.wikidata.org/entity/Q1074654'), ('label', "2006 Wimbledon Championships – Men's Singles")]
[('object', 'http://www.wikidata.org/entity/Q384929'), ('label', "2005 Wimbledon Championships – Men's Singles")]
[('object', 'h

In [40]:
# getting object/data properties of node '2006 wimbledon championships – men's singles'
queryString = """
SELECT DISTINCT ?property ?label
WHERE
{
   wd:Q1074654 ?property ?object.
   
   ?property sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1346'), ('label', 'winner')]
[('property', 'http://www.wikidata.org/prop/direct/P155'), ('label', 'follows')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('label', 'country')]
[('property', 'http://www.wikidata.org/prop/direct/P2094'), ('label', 'competition class')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P361'), ('label', 'part of')]
[('property', 'http://www.wikidata.org/prop/direct/P585'), ('label', 'point in time')]
[('property', 'http://www.wikidata.org/prop/direct/P641'), ('label', 'sport')]
[('property', 'http://www.wikidata.org/prop/direct/P646'), ('label', 'Freebase ID')]
9


In [41]:
# s:what node? p:'instance of' o:'wimbledon chamiponships'
queryString = """
SELECT ?subject ?label
WHERE
{
   ?subject wdt:P31 wd:Q41520.
   
   ?subject sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('subject', 'http://www.wikidata.org/entity/Q268871'), ('label', '2004 Wimbledon Championships')]
[('subject', 'http://www.wikidata.org/entity/Q173961'), ('label', '1978 Wimbledon Championships')]
[('subject', 'http://www.wikidata.org/entity/Q925026'), ('label', '1971 Wimbledon Championships')]
[('subject', 'http://www.wikidata.org/entity/Q15854765'), ('label', '2014 Wimbledon Championships')]
[('subject', 'http://www.wikidata.org/entity/Q30085309'), ('label', '2018 Wimbledon Championships')]
[('subject', 'http://www.wikidata.org/entity/Q745901'), ('label', '1974 Wimbledon Championships')]
[('subject', 'http://www.wikidata.org/entity/Q655368'), ('label', '1955 Wimbledon Championships')]
[('subject', 'http://www.wikidata.org/entity/Q655378'), ('label', '1956 Wimbledon Championships')]
[('subject', 'http://www.wikidata.org/entity/Q657950'), ('label', '1957 Wimbledon Championships')]
[('subject', 'http://www.wikidata.org/entity/Q657962'), ('label', '1958 Wimbledon Championships')

In [42]:
# s:what node? p:'part of' o:'2018 wimbledon chamiponships'
queryString = """
SELECT ?subject ?label
WHERE
{
   ?subject wdt:P361 wd:Q30085309.
   
   ?subject sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('subject', 'http://www.wikidata.org/entity/Q30098268'), ('label', "2018 Wimbledon Championships – Women's Singles")]
[('subject', 'http://www.wikidata.org/entity/Q32358401'), ('label', '2018 Wimbledon Championships – Mixed Doubles')]
[('subject', 'http://www.wikidata.org/entity/Q55639017'), ('label', '2018 Wimbledon Championships – Wheelchair Quad Doubles')]
[('subject', 'http://www.wikidata.org/entity/Q55455418'), ('label', "2018 Wimbledon Championships – Wheelchair Men's Doubles")]
[('subject', 'http://www.wikidata.org/entity/Q32357790'), ('label', "2018 Wimbledon Championships – Men's Doubles")]
[('subject', 'http://www.wikidata.org/entity/Q32356704'), ('label', "2018 Wimbledon Championships – Women's Doubles")]
[('subject', 'http://www.wikidata.org/entity/Q55381839'), ('label', "2018 Wimbledon Championships – Girls' Singles")]
[('subject', 'http://www.wikidata.org/entity/Q54961494'), ('label', "2018 Wimbledon Championships – Men's Singles")]
[('subject', 'http://www.wikid

In [43]:
# checking 'winner' property of node '2018 wimbledon championships – women's singles'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q30098268 wdt:P1346 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q77178'), ('label', 'Angelique Kerber')]
1


Final query for this task

In [44]:
# write your final query
queryString = """
SELECT ?tournament ?label_tournament ?winner ?label_winner
WHERE
{
   ?tournament wdt:P361 wd:Q30085309.
   ?tournament wdt:P1346 ?winner.
   
   ?tournament sc:name ?label_tournament.
   ?winner sc:name ?label_winner.
}
"""

print("Results")
x=run_query(queryString)

Results
[('tournament', 'http://www.wikidata.org/entity/Q32357790'), ('label_tournament', "2018 Wimbledon Championships – Men's Doubles"), ('winner', 'http://www.wikidata.org/entity/Q53342'), ('label_winner', 'Mike Bryan')]
[('tournament', 'http://www.wikidata.org/entity/Q54961494'), ('label_tournament', "2018 Wimbledon Championships – Men's Singles"), ('winner', 'http://www.wikidata.org/entity/Q5812'), ('label_winner', 'Novak Djokovic')]
[('tournament', 'http://www.wikidata.org/entity/Q30098268'), ('label_tournament', "2018 Wimbledon Championships – Women's Singles"), ('winner', 'http://www.wikidata.org/entity/Q77178'), ('label_winner', 'Angelique Kerber')]
[('tournament', 'http://www.wikidata.org/entity/Q32356704'), ('label_tournament', "2018 Wimbledon Championships – Women's Doubles"), ('winner', 'http://www.wikidata.org/entity/Q11052500'), ('label_winner', 'Barbora Krejčíková')]
[('tournament', 'http://www.wikidata.org/entity/Q55455423'), ('label_tournament', "2018 Wimbledon Champi

## Task 6
Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

In [45]:
# write your queries
# getting tennis players who has 'date of birth' property after 1975
queryString = """
SELECT ?player ?label ?date
WHERE
{
   ?player wdt:P106 wd:Q10833314.
   ?player wdt:P569 ?date.
   
   FILTER (datatype(?date) = xsd:dateTime)
   FILTER ("1975-01-01T00:00:00Z"^^xsd:dateTime <= ?date)
   
   ?player sc:name ?label.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q138094'), ('label', 'Anna Karolína Schmiedlová'), ('date', '1994-09-13T00:00:00Z')]
[('player', 'http://www.wikidata.org/entity/Q15197746'), ('label', 'Wesley Koolhof'), ('date', '1989-04-17T00:00:00Z')]
[('player', 'http://www.wikidata.org/entity/Q15880574'), ('label', 'Paul Oosterbaan'), ('date', '1995-07-08T00:00:00Z')]
[('player', 'http://www.wikidata.org/entity/Q240837'), ('label', 'Rika Fujiwara'), ('date', '1981-09-19T00:00:00Z')]
[('player', 'http://www.wikidata.org/entity/Q4134097'), ('label', 'Margarita Gasparyan'), ('date', '1994-09-01T00:00:00Z')]
[('player', 'http://www.wikidata.org/entity/Q16224631'), ('label', 'Lee So-ra'), ('date', '1994-07-22T00:00:00Z')]
[('player', 'http://www.wikidata.org/entity/Q7281705'), ('label', 'Radu Albot'), ('date', '1989-11-11T00:00:00Z')]
[('player', 'http://www.wikidata.org/entity/Q18918709'), ('label', 'Markéta Vondroušová'), ('date', '1999-06-28T00:00:00Z')]
[('player', 'http://www.wi

In [46]:
# getting all editions of all grand slams with the iri of each grand slam separately
queryString = """
SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
WHERE
{
   values ?grand_slam { wd:Q60874 wd:Q43605 wd:Q41520 wd:Q123577 }
   
   ?yearWise_edition_of_grand_slam wdt:P31 ?grand_slam.
   ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.
   
   ?eventWise_edition_of_grand_slam sc:name ?label.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('eventWise_edition_of_grand_slam', 'http://www.wikidata.org/entity/Q15851905'), ('label', "1970 US Open – Men's Doubles")]
[('eventWise_edition_of_grand_slam', 'http://www.wikidata.org/entity/Q15854748'), ('label', "1971 Wimbledon Championships – Men's Doubles")]
[('eventWise_edition_of_grand_slam', 'http://www.wikidata.org/entity/Q15854755'), ('label', "1978 Wimbledon Championships – Men's Doubles")]
[('eventWise_edition_of_grand_slam', 'http://www.wikidata.org/entity/Q15854764'), ('label', "2012 Wimbledon Championships – Ladies' Invitation Doubles")]
[('eventWise_edition_of_grand_slam', 'http://www.wikidata.org/entity/Q16040928'), ('label', "1928 French Championships – Women's Doubles")]
[('eventWise_edition_of_grand_slam', 'http://www.wikidata.org/entity/Q16040959'), ('label', "1938 French Championships – Women's Doubles")]
[('eventWise_edition_of_grand_slam', 'http://www.wikidata.org/entity/Q17016282'), ('label', "2014 French Open – Wheelchair Men's Singles")]
[('eventWis

In [47]:
# counting all victories of top 10 players
queryString = """
SELECT ?player ?label (COUNT(?victory) AS ?num_victory)
WHERE
{
    ?player wdt:P106 wd:Q10833314.
    ?player wdt:P2522 ?victory.
    
    ?player sc:name ?label.
}
GROUP BY ?player ?label
ORDER BY DESC(?num_victory)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q1426'), ('label', 'Roger Federer'), ('num_victory', '29')]
[('player', 'http://www.wikidata.org/entity/Q202342'), ('label', 'Margaret Court'), ('num_victory', '24')]
[('player', 'http://www.wikidata.org/entity/Q54543'), ('label', 'Helen Wills'), ('num_victory', '19')]
[('player', 'http://www.wikidata.org/entity/Q54527'), ('label', 'Billie Jean King'), ('num_victory', '12')]
[('player', 'http://www.wikidata.org/entity/Q53390'), ('label', 'Maureen Connolly'), ('num_victory', '9')]
[('player', 'http://www.wikidata.org/entity/Q34276'), ('label', 'Garbiñe Muguruza'), ('num_victory', '8')]
[('player', 'http://www.wikidata.org/entity/Q233247'), ('label', 'Maria Bueno'), ('num_victory', '7')]
[('player', 'http://www.wikidata.org/entity/Q11662'), ('label', 'Steffi Graf'), ('num_victory', '6')]
[('player', 'http://www.wikidata.org/entity/Q53564'), ('label', 'Doris Hart'), ('num_victory', '6')]
[('player', 'http://www.wikidata.org/entity/Q48351

Final query for this task

In [48]:
# write your final query
queryString = """
SELECT ?player ?label_player (COUNT(*) AS ?num_victory)
WHERE
{
    ?player wdt:P106 wd:Q10833314.
    ?player wdt:P2522 ?victory.
    
    FILTER (?victory = ?eventWise_edition_of_grand_slam)
    
    ?player sc:name ?label_player.
    {
        SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
        WHERE
        {
           values ?grand_slam { wd:Q60874 wd:Q43605 wd:Q41520 wd:Q123577 }

           ?yearWise_edition_of_grand_slam wdt:P31 ?grand_slam.
           ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.

           ?eventWise_edition_of_grand_slam sc:name ?label.
        }
    }
    
}
GROUP BY ?player ?label_player 
ORDER BY DESC(?num_victory)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q202342'), ('label_player', 'Margaret Court'), ('num_victory', '24')]
[('player', 'http://www.wikidata.org/entity/Q1426'), ('label_player', 'Roger Federer'), ('num_victory', '20')]
[('player', 'http://www.wikidata.org/entity/Q54543'), ('label_player', 'Helen Wills'), ('num_victory', '19')]
[('player', 'http://www.wikidata.org/entity/Q54527'), ('label_player', 'Billie Jean King'), ('num_victory', '12')]
[('player', 'http://www.wikidata.org/entity/Q53390'), ('label_player', 'Maureen Connolly'), ('num_victory', '9')]
[('player', 'http://www.wikidata.org/entity/Q233247'), ('label_player', 'Maria Bueno'), ('num_victory', '7')]
[('player', 'http://www.wikidata.org/entity/Q11662'), ('label_player', 'Steffi Graf'), ('num_victory', '6')]
[('player', 'http://www.wikidata.org/entity/Q53564'), ('label_player', 'Doris Hart'), ('num_victory', '6')]
[('player', 'http://www.wikidata.org/entity/Q48351'), ('label_player', 'Dorothy Round'), ('num_victor

## Task 7
Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

In [49]:
# write your queries
# checking 'country for sport' property of node 'Roger Federer'
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q1426 wdt:P1532 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q39'), ('label', 'Switzerland')]
1


In [50]:
# getting all tennis players
queryString = """
SELECT DISTINCT ?subject ?label
WHERE
{
   ?subject wdt:P106 wd:Q10833314.
   
   ?subject sc:name ?label.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('subject', 'http://www.wikidata.org/entity/Q1683632'), ('label', 'Jaroslav Navrátil')]
[('subject', 'http://www.wikidata.org/entity/Q138094'), ('label', 'Anna Karolína Schmiedlová')]
[('subject', 'http://www.wikidata.org/entity/Q15197746'), ('label', 'Wesley Koolhof')]
[('subject', 'http://www.wikidata.org/entity/Q1558013'), ('label', 'Pierre Barthès')]
[('subject', 'http://www.wikidata.org/entity/Q15849678'), ('label', 'Sue Partridge')]
[('subject', 'http://www.wikidata.org/entity/Q15880574'), ('label', 'Paul Oosterbaan')]
[('subject', 'http://www.wikidata.org/entity/Q240837'), ('label', 'Rika Fujiwara')]
[('subject', 'http://www.wikidata.org/entity/Q4134097'), ('label', 'Margarita Gasparyan')]
[('subject', 'http://www.wikidata.org/entity/Q589611'), ('label', 'Philippe Chatrier')]
[('subject', 'http://www.wikidata.org/entity/Q16009513'), ('label', 'James Bayley')]
[('subject', 'http://www.wikidata.org/entity/Q16023406'), ('label', 'Samuel T. Chase')]
[('subject', 'http://www

In [51]:
# getting 'country for sport' property of all players
queryString = """
SELECT DISTINCT ?object ?label
WHERE
{
   ?player wdt:P106 wd:Q10833314.
   ?player wdt:P1532 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q216923'), ('label', 'Chinese Taipei')]
[('object', 'http://www.wikidata.org/entity/Q1019'), ('label', 'Madagascar')]
[('object', 'http://www.wikidata.org/entity/Q129286'), ('label', 'British India')]
[('object', 'http://www.wikidata.org/entity/Q142'), ('label', 'France')]
[('object', 'http://www.wikidata.org/entity/Q145'), ('label', 'United Kingdom')]
[('object', 'http://www.wikidata.org/entity/Q148'), ('label', "People's Republic of China")]
[('object', 'http://www.wikidata.org/entity/Q15180'), ('label', 'Soviet Union')]
[('object', 'http://www.wikidata.org/entity/Q155'), ('label', 'Brazil')]
[('object', 'http://www.wikidata.org/entity/Q159'), ('label', 'Russia')]
[('object', 'http://www.wikidata.org/entity/Q16'), ('label', 'Canada')]
[('object', 'http://www.wikidata.org/entity/Q16957'), ('label', 'German Democratic Republic')]
[('object', 'http://www.wikidata.org/entity/Q17'), ('label', 'Japan')]
[('object', 'http://www.wikidata.or

In [52]:
# getting all players that won an edition of 'australian open'
queryString = """
SELECT ?player ?label_player
WHERE
{
    ?player wdt:P106 wd:Q10833314.
    ?player wdt:P2522 ?victory.
    
    FILTER (?victory = ?eventWise_edition_of_grand_slam)
    
    ?player sc:name ?label_player.
    {
        SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
        WHERE
        {
           ?yearWise_edition_of_grand_slam wdt:P31 wd:Q60874.
           ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.

           ?eventWise_edition_of_grand_slam sc:name ?label.
        }
    }
}
GROUP BY ?player ?label_player 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q10118'), ('label_player', 'Victoria Azarenka')]
[('player', 'http://www.wikidata.org/entity/Q84079953'), ('label_player', 'Nicholas David Ionel')]
[('player', 'http://www.wikidata.org/entity/Q53424'), ('label_player', 'Shirley Fry')]
[('player', 'http://www.wikidata.org/entity/Q53390'), ('label_player', 'Maureen Connolly')]
[('player', 'http://www.wikidata.org/entity/Q54527'), ('label_player', 'Billie Jean King')]
[('player', 'http://www.wikidata.org/entity/Q131120'), ('label_player', 'Anna Kournikova')]
[('player', 'http://www.wikidata.org/entity/Q17466583'), ('label_player', 'Naomi Osaka')]
[('player', 'http://www.wikidata.org/entity/Q202342'), ('label_player', 'Margaret Court')]
[('player', 'http://www.wikidata.org/entity/Q54546'), ('label_player', 'Nancy Richey')]
[('player', 'http://www.wikidata.org/entity/Q53338'), ('label_player', 'Louise Brough')]
[('player', 'http://www.wikidata.org/entity/Q11677'), ('label_player', 'Lindsay

In [53]:
# getting all players that won an edition of 'french open'
queryString = """
SELECT ?player ?label_player 
WHERE
{
    ?player wdt:P106 wd:Q10833314.
    ?player wdt:P2522 ?victory.
    
    FILTER (?victory = ?eventWise_edition_of_grand_slam)
    
    ?player sc:name ?label_player.
    {
        SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
        WHERE
        {
           ?yearWise_edition_of_grand_slam wdt:P31 wd:Q43605.
           ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.

           ?eventWise_edition_of_grand_slam sc:name ?label.
        }
    }
}
GROUP BY ?player ?label_player 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q106354932'), ('label_player', 'Linda Nosková')]
[('player', 'http://www.wikidata.org/entity/Q53424'), ('label_player', 'Shirley Fry')]
[('player', 'http://www.wikidata.org/entity/Q53390'), ('label_player', 'Maureen Connolly')]
[('player', 'http://www.wikidata.org/entity/Q54543'), ('label_player', 'Helen Wills')]
[('player', 'http://www.wikidata.org/entity/Q54527'), ('label_player', 'Billie Jean King')]
[('player', 'http://www.wikidata.org/entity/Q11581'), ('label_player', 'Andrea Sestini Hlaváčková')]
[('player', 'http://www.wikidata.org/entity/Q53550'), ('label_player', 'Althea Gibson')]
[('player', 'http://www.wikidata.org/entity/Q106113'), ('label_player', 'Ilie Năstase')]
[('player', 'http://www.wikidata.org/entity/Q202342'), ('label_player', 'Margaret Court')]
[('player', 'http://www.wikidata.org/entity/Q1144091'), ('label_player', 'Hank Pfister')]
[('player', 'http://www.wikidata.org/entity/Q54546'), ('label_player', 'Nancy Ric

In [54]:
# getting all players that won an edition of 'wimbledon championships'
queryString = """
SELECT ?player ?label_player
WHERE
{
    ?player wdt:P106 wd:Q10833314.
    ?player wdt:P2522 ?victory.
    
    FILTER (?victory = ?eventWise_edition_of_grand_slam)
    
    ?player sc:name ?label_player.
    {
        SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
        WHERE
        {
           ?yearWise_edition_of_grand_slam wdt:P31 wd:Q41520.
           ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.

           ?eventWise_edition_of_grand_slam sc:name ?label.
        }
    }
}
GROUP BY ?player ?label_player 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q10125'), ('label_player', 'Andy Murray')]
[('player', 'http://www.wikidata.org/entity/Q233247'), ('label_player', 'Maria Bueno')]
[('player', 'http://www.wikidata.org/entity/Q233234'), ('label_player', 'Virginia Wade')]
[('player', 'http://www.wikidata.org/entity/Q53424'), ('label_player', 'Shirley Fry')]
[('player', 'http://www.wikidata.org/entity/Q53390'), ('label_player', 'Maureen Connolly')]
[('player', 'http://www.wikidata.org/entity/Q54543'), ('label_player', 'Helen Wills')]
[('player', 'http://www.wikidata.org/entity/Q54527'), ('label_player', 'Billie Jean King')]
[('player', 'http://www.wikidata.org/entity/Q53550'), ('label_player', 'Althea Gibson')]
[('player', 'http://www.wikidata.org/entity/Q105550'), ('label_player', 'Rod Laver')]
[('player', 'http://www.wikidata.org/entity/Q202342'), ('label_player', 'Margaret Court')]
[('player', 'http://www.wikidata.org/entity/Q53567'), ('label_player', 'Helen Jacobs')]
[('player', 'ht

In [55]:
# getting all players that won an edition of 'US open'
queryString = """
SELECT ?player ?label_player
WHERE
{
    ?player wdt:P106 wd:Q10833314.
    ?player wdt:P2522 ?victory.
    
    FILTER (?victory = ?eventWise_edition_of_grand_slam)
    
    ?player sc:name ?label_player.
    {
        SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
        WHERE
        {
           ?yearWise_edition_of_grand_slam wdt:P31 wd:Q123577.
           ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.

           ?eventWise_edition_of_grand_slam sc:name ?label.
        }
    }
}
GROUP BY ?player ?label_player 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q10125'), ('label_player', 'Andy Murray')]
[('player', 'http://www.wikidata.org/entity/Q233247'), ('label_player', 'Maria Bueno')]
[('player', 'http://www.wikidata.org/entity/Q53424'), ('label_player', 'Shirley Fry')]
[('player', 'http://www.wikidata.org/entity/Q53390'), ('label_player', 'Maureen Connolly')]
[('player', 'http://www.wikidata.org/entity/Q54543'), ('label_player', 'Helen Wills')]
[('player', 'http://www.wikidata.org/entity/Q54527'), ('label_player', 'Billie Jean King')]
[('player', 'http://www.wikidata.org/entity/Q53550'), ('label_player', 'Althea Gibson')]
[('player', 'http://www.wikidata.org/entity/Q106113'), ('label_player', 'Ilie Năstase')]
[('player', 'http://www.wikidata.org/entity/Q17466583'), ('label_player', 'Naomi Osaka')]
[('player', 'http://www.wikidata.org/entity/Q202342'), ('label_player', 'Margaret Court')]
[('player', 'http://www.wikidata.org/entity/Q53567'), ('label_player', 'Helen Jacobs')]
[('player', 

In [56]:
# getting all tennis players that won an edition of each grand slam
queryString = """
SELECT * 
WHERE
{ 
    {
        SELECT ?player ?label_player 
        WHERE
        {
            ?player wdt:P106 wd:Q10833314.
            ?player wdt:P2522 ?victory.

            FILTER (?victory = ?eventWise_edition_of_grand_slam)

            ?player sc:name ?label_player.
            {
                SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
                WHERE
                {
                   ?yearWise_edition_of_grand_slam wdt:P31 wd:Q60874.
                   ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.

                   ?eventWise_edition_of_grand_slam sc:name ?label.
                }
            }
        }
        GROUP BY ?player ?label_player 
    }
    UNION
    {
        SELECT ?player ?label_player
        WHERE
        {
            ?player wdt:P106 wd:Q10833314.
            ?player wdt:P2522 ?victory.

            FILTER (?victory = ?eventWise_edition_of_grand_slam)

            ?player sc:name ?label_player.
            {
                SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
                WHERE
                {
                   ?yearWise_edition_of_grand_slam wdt:P31 wd:Q43605.
                   ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.

                   ?eventWise_edition_of_grand_slam sc:name ?label.
                }
            }
        }
        GROUP BY ?player ?label_player 
    }
    UNION
    {
        SELECT ?player ?label_player 
        WHERE
        {
            ?player wdt:P106 wd:Q10833314.
            ?player wdt:P2522 ?victory.

            FILTER (?victory = ?eventWise_edition_of_grand_slam)

            ?player sc:name ?label_player.
            {
                SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
                WHERE
                {
                   ?yearWise_edition_of_grand_slam wdt:P31 wd:Q41520.
                   ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.

                   ?eventWise_edition_of_grand_slam sc:name ?label.
                }
            }
        }
        GROUP BY ?player ?label_player 
    }
    UNION
    {
        SELECT DISTINCT ?player ?label_player
        WHERE
        {
            ?player wdt:P106 wd:Q10833314.
            ?player wdt:P2522 ?victory.

            FILTER (?victory = ?eventWise_edition_of_grand_slam)

            ?player sc:name ?label_player.
            {
                SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
                WHERE
                {
                   ?yearWise_edition_of_grand_slam wdt:P31 wd:Q123577.
                   ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.

                   ?eventWise_edition_of_grand_slam sc:name ?label.
                }
            }
        }
        GROUP BY ?player ?label_player 
    }
}
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q54546'), ('label_player', 'Nancy Richey')]
[('player', 'http://www.wikidata.org/entity/Q202342'), ('label_player', 'Margaret Court')]
[('player', 'http://www.wikidata.org/entity/Q10118'), ('label_player', 'Victoria Azarenka')]
[('player', 'http://www.wikidata.org/entity/Q53424'), ('label_player', 'Shirley Fry')]
[('player', 'http://www.wikidata.org/entity/Q53390'), ('label_player', 'Maureen Connolly')]
[('player', 'http://www.wikidata.org/entity/Q53564'), ('label_player', 'Doris Hart')]
[('player', 'http://www.wikidata.org/entity/Q131120'), ('label_player', 'Anna Kournikova')]
[('player', 'http://www.wikidata.org/entity/Q1426'), ('label_player', 'Roger Federer')]
[('player', 'http://www.wikidata.org/entity/Q54527'), ('label_player', 'Billie Jean King')]
[('player', 'http://www.wikidata.org/entity/Q53338'), ('label_player', 'Louise Brough')]
[('player', 'http://www.wikidata.org/entity/Q11677'), ('label_player', 'Lindsay Davenport')]
[

In [57]:
# getting the players from last query that has been reported 4 times meaning
# that they have won at least one edition of each grand slam

# to me at least one edition of EACH grand slam means that a player who has won at least once in ALL 4 grand slams
queryString = """
SELECT ?player ?label_player (COUNT (*) AS ?count)
WHERE
{ 
    {
        SELECT ?player ?label_player
        WHERE
        {
            ?player wdt:P106 wd:Q10833314.
            ?player wdt:P2522 ?victory.

            FILTER (?victory = ?eventWise_edition_of_grand_slam)

            ?player sc:name ?label_player.
            {
                SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
                WHERE
                {
                   ?yearWise_edition_of_grand_slam wdt:P31 wd:Q60874.
                   ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.

                   ?eventWise_edition_of_grand_slam sc:name ?label.
                }
            }
        }
        GROUP BY ?player ?label_player 
    }
    UNION
    {
        SELECT ?player ?label_player 
        WHERE
        {
            ?player wdt:P106 wd:Q10833314.
            ?player wdt:P2522 ?victory.

            FILTER (?victory = ?eventWise_edition_of_grand_slam)

            ?player sc:name ?label_player.
            {
                SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
                WHERE
                {
                   ?yearWise_edition_of_grand_slam wdt:P31 wd:Q43605.
                   ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.

                   ?eventWise_edition_of_grand_slam sc:name ?label.
                }
            }
        }
        GROUP BY ?player ?label_player 
    }
    UNION
    {
        SELECT ?player ?label_player
        WHERE
        {
            ?player wdt:P106 wd:Q10833314.
            ?player wdt:P2522 ?victory.

            FILTER (?victory = ?eventWise_edition_of_grand_slam)

            ?player sc:name ?label_player.
            {
                SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
                WHERE
                {
                   ?yearWise_edition_of_grand_slam wdt:P31 wd:Q41520.
                   ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.

                   ?eventWise_edition_of_grand_slam sc:name ?label.
                }
            }
        }
        GROUP BY ?player ?label_player 
    }
    UNION
    {
        SELECT DISTINCT ?player ?label_player
        WHERE
        {
            ?player wdt:P106 wd:Q10833314.
            ?player wdt:P2522 ?victory.

            FILTER (?victory = ?eventWise_edition_of_grand_slam)

            ?player sc:name ?label_player.
            {
                SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
                WHERE
                {
                   ?yearWise_edition_of_grand_slam wdt:P31 wd:Q123577.
                   ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.

                   ?eventWise_edition_of_grand_slam sc:name ?label.
                }
            }
        }
        GROUP BY ?player ?label_player 
    }
}
GROUP BY ?player ?label_player
HAVING (COUNT(*) = 4)
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q53424'), ('label_player', 'Shirley Fry'), ('count', '4')]
[('player', 'http://www.wikidata.org/entity/Q53390'), ('label_player', 'Maureen Connolly'), ('count', '4')]
[('player', 'http://www.wikidata.org/entity/Q54527'), ('label_player', 'Billie Jean King'), ('count', '4')]
[('player', 'http://www.wikidata.org/entity/Q202342'), ('label_player', 'Margaret Court'), ('count', '4')]
[('player', 'http://www.wikidata.org/entity/Q53564'), ('label_player', 'Doris Hart'), ('count', '4')]
[('player', 'http://www.wikidata.org/entity/Q1426'), ('label_player', 'Roger Federer'), ('count', '4')]
6


In [58]:
# checking 'country for sport' property of node 'Margaret Court'

# the reason for checking this query was that when I included ?player wdt:P1532 ?country in the last query's
# clauses, it affected the output results and I did not get 'Margaret Court' as an answer. The snapshot seems  
# again a bit problematic because P1532 actually exists for 'Margaret Court' on wikidata online
queryString = """
SELECT ?object ?label
WHERE
{
   wd:Q202342 wdt:P1532 ?object.
   
   ?object sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
Empty


Final query for this task

In [59]:
# write your final query

# as in the last query, after using P1532 property, 'Margaret Court' gets excluded from the result set because
# of the mentioned problem and so, country 'australia' is not shown in the final output
queryString = """
SELECT DISTINCT ?country ?label
WHERE
{
    ?player wdt:P1532 ?country.
    
    ?country sc:name ?label.
    {
        SELECT ?player ?label_player (COUNT (*) AS ?count)
        WHERE
        { 
            {
                SELECT ?player ?label_player
                WHERE
                {
                    ?player wdt:P106 wd:Q10833314.
                    ?player wdt:P2522 ?victory.

                    FILTER (?victory = ?eventWise_edition_of_grand_slam)

                    ?player sc:name ?label_player.
                    {
                        SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
                        WHERE
                        {
                           ?yearWise_edition_of_grand_slam wdt:P31 wd:Q60874.
                           ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.

                           ?eventWise_edition_of_grand_slam sc:name ?label.
                        }
                    }
                }
                GROUP BY ?player ?label_player 
            }
            UNION
            {
                SELECT ?player ?label_player
                WHERE
                {
                    ?player wdt:P106 wd:Q10833314.
                    ?player wdt:P2522 ?victory.

                    FILTER (?victory = ?eventWise_edition_of_grand_slam)

                    ?player sc:name ?label_player.
                    {
                        SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
                        WHERE
                        {
                           ?yearWise_edition_of_grand_slam wdt:P31 wd:Q43605.
                           ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.

                           ?eventWise_edition_of_grand_slam sc:name ?label.
                        }
                    }
                }
                GROUP BY ?player ?label_player 
            }
            UNION
            {
                SELECT ?player ?label_player
                WHERE
                {
                    ?player wdt:P106 wd:Q10833314.
                    ?player wdt:P2522 ?victory.

                    FILTER (?victory = ?eventWise_edition_of_grand_slam)

                    ?player sc:name ?label_player.
                    {
                        SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
                        WHERE
                        {
                           ?yearWise_edition_of_grand_slam wdt:P31 wd:Q41520.
                           ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.

                           ?eventWise_edition_of_grand_slam sc:name ?label.
                        }
                    }
                }
                GROUP BY ?player ?label_player 
            }
            UNION
            {
                SELECT DISTINCT ?player ?label_player
                WHERE
                {
                    ?player wdt:P106 wd:Q10833314.
                    ?player wdt:P2522 ?victory.

                    FILTER (?victory = ?eventWise_edition_of_grand_slam)

                    ?player sc:name ?label_player.
                    {
                        SELECT DISTINCT ?eventWise_edition_of_grand_slam ?label
                        WHERE
                        {
                           ?yearWise_edition_of_grand_slam wdt:P31 wd:Q123577.
                           ?eventWise_edition_of_grand_slam wdt:P361 ?yearWise_edition_of_grand_slam.

                           ?eventWise_edition_of_grand_slam sc:name ?label.
                        }
                    }
                }
                GROUP BY ?player ?label_player 
            }
        }
        GROUP BY ?player ?label_player
        HAVING (COUNT(*) = 4)
    }
}
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q30'), ('label', 'United States of America')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('label', 'Switzerland')]
2


## Task 8
Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

In [60]:
# write your queries
# checking the 'ranking' property of node 'Roger Federer' to see if is 1
queryString = """
SELECT ?object
WHERE
{
   wd:Q1426 wdt:P1352 ?object.

   FILTER (datatype(?object) = xsd:int)
   FILTER("1"^^xsd:int = ?object)
}
"""

print("Results")
x=run_query(queryString)

Results
[('object', '1')]
1


In [61]:
# getting all the players who ranked 1
queryString = """
SELECT ?player ?label ?ranking
WHERE
{
   ?player wdt:P106 wd:Q10833314.
   ?player wdt:P1352 ?ranking.

   FILTER (datatype(?ranking) = xsd:int)
   FILTER("1"^^xsd:int = ?ranking)
   
   ?player sc:name ?label.
}
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q207352'), ('label', 'Agnieszka Radwańska'), ('ranking', '1')]
[('player', 'http://www.wikidata.org/entity/Q455136'), ('label', 'Anders Järryd'), ('ranking', '1')]
[('player', 'http://www.wikidata.org/entity/Q455411'), ('label', 'Tom Okker'), ('ranking', '1')]
[('player', 'http://www.wikidata.org/entity/Q310419'), ('label', 'Nenad Zimonjić'), ('ranking', '1')]
[('player', 'http://www.wikidata.org/entity/Q53393'), ('label', 'Jimmy Connors'), ('ranking', '1')]
[('player', 'http://www.wikidata.org/entity/Q10118'), ('label', 'Victoria Azarenka'), ('ranking', '1')]
[('player', 'http://www.wikidata.org/entity/Q13538'), ('label', 'Henri Kontinen'), ('ranking', '1')]
[('player', 'http://www.wikidata.org/entity/Q1743002'), ('label', 'Robert Seguso'), ('ranking', '1')]
[('player', 'http://www.wikidata.org/entity/Q189542'), ('label', 'Stefan Edberg'), ('ranking', '1')]
[('player', 'http://www.wikidata.org/entity/Q232566'), ('label', 'Hsieh Su-we

In [62]:
# getting the countries of all the players who ranked 1
queryString = """
SELECT ?player ?ranking ?label
WHERE
{
   ?player wdt:P106 wd:Q10833314.
   ?player wdt:P1352 ?ranking.
   ?player wdt:P1532 ?country.

   FILTER (datatype(?ranking) = xsd:int)
   FILTER("1"^^xsd:int = ?ranking)
   
   ?country sc:name ?label
}
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q232566'), ('ranking', '1'), ('label', 'Chinese Taipei')]
[('player', 'http://www.wikidata.org/entity/Q234624'), ('ranking', '1'), ('label', 'Chinese Taipei')]
[('player', 'http://www.wikidata.org/entity/Q221908'), ('ranking', '1'), ('label', 'France')]
[('player', 'http://www.wikidata.org/entity/Q180872'), ('ranking', '1'), ('label', 'France')]
[('player', 'http://www.wikidata.org/entity/Q241320'), ('ranking', '1'), ('label', 'France')]
[('player', 'http://www.wikidata.org/entity/Q242959'), ('ranking', '1'), ('label', 'France')]
[('player', 'http://www.wikidata.org/entity/Q317974'), ('ranking', '1'), ('label', 'France')]
[('player', 'http://www.wikidata.org/entity/Q503183'), ('ranking', '1'), ('label', 'United Kingdom')]
[('player', 'http://www.wikidata.org/entity/Q229087'), ('ranking', '1'), ('label', "People's Republic of China")]
[('player', 'http://www.wikidata.org/entity/Q41799108'), ('ranking', '1'), ('label', "People's Republi

Final query for this task

In [63]:
# write your final query
queryString = """
SELECT DISTINCT ?country_iri ?label
WHERE
{
    SELECT ?player ?ranking ?country_iri ?label
    WHERE
    {
       ?player wdt:P106 wd:Q10833314.
       ?player wdt:P1352 ?ranking.
       ?player wdt:P1532 ?country_iri.

       FILTER (datatype(?ranking) = xsd:int)
       FILTER("1"^^xsd:int = ?ranking)

       ?country_iri sc:name ?label
    }
}
"""

print("Results")
x=run_query(queryString)

Results
[('country_iri', 'http://www.wikidata.org/entity/Q216923'), ('label', 'Chinese Taipei')]
[('country_iri', 'http://www.wikidata.org/entity/Q142'), ('label', 'France')]
[('country_iri', 'http://www.wikidata.org/entity/Q145'), ('label', 'United Kingdom')]
[('country_iri', 'http://www.wikidata.org/entity/Q148'), ('label', "People's Republic of China")]
[('country_iri', 'http://www.wikidata.org/entity/Q15180'), ('label', 'Soviet Union')]
[('country_iri', 'http://www.wikidata.org/entity/Q155'), ('label', 'Brazil')]
[('country_iri', 'http://www.wikidata.org/entity/Q159'), ('label', 'Russia')]
[('country_iri', 'http://www.wikidata.org/entity/Q16'), ('label', 'Canada')]
[('country_iri', 'http://www.wikidata.org/entity/Q17'), ('label', 'Japan')]
[('country_iri', 'http://www.wikidata.org/entity/Q183'), ('label', 'Germany')]
[('country_iri', 'http://www.wikidata.org/entity/Q184'), ('label', 'Belarus')]
[('country_iri', 'http://www.wikidata.org/entity/Q191'), ('label', 'Estonia')]
[('countr